## Project Title

### Project Description

Our team will focus on studying the relationship (if any) of the average land temperature (i.e., #2 proposed dataset) and the evolution of population in such land, along with its GDP. Datasets will be pulled from the World Bank and are available in `.csv`, `.xml` and `.xlsx` formats in the following links, respectively:

[Population](https://data.worldbank.org/indicator/SP.POP.TOTL)

[GDP (2015 USD)](https://data.worldbank.org/indicator/NY.GDP.MKTP.KD)

Behavior will be analysed mainly by visual means (such as scatterplots, pyramid populations, etc.) and reported in Markdown in the corresponding .ipynb file. 

> Imported libraries

In [196]:
import pandas as pd # For data manipulation
import geopandas # For map visualization
import folium # For map visualization
import pycountry # For inconsistent country names
import numpy as np # linear algebra

> Previous calculations

Create a dictionary of inconsistent country names and their standardized names:

In [197]:
country_dict = {}
for country in pycountry.countries:
    country_dict[country.name] = country.name
    country_dict[country.alpha_2] = country.name
    country_dict[country.alpha_3] = country.name

---

## Programming Component


### Reading and cleaning the main dataset


In [198]:
df = pd.read_csv('..\data\land-data\GlobalLandTemperaturesByCountry.csv') # Reading GlobalLandTemperaturesByCountry csv file
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
1,1743-12-01,NaN,NaN,Åland
2,1744-01-01,NaN,NaN,Åland
3,1744-02-01,NaN,NaN,Åland
4,1744-03-01,NaN,NaN,Åland


Next setp will be convert `dt` colum in to a date type one and we will only be focusing in the `year` information. 

In [199]:
df.dt = pd.to_datetime(df.dt) # Convert to datetime
df.dt = df.dt.dt.year # Extract year

In [200]:
df.rename(columns={"dt": "Year"}, inplace=True) # Rename column
df_year_temp =df.groupby(['Country', 'Year']).mean() # Group by country and year
df_year_temp.drop(columns=['AverageTemperatureUncertainty'], inplace=True) # Drop unnecessary column
df_year_temp.head() 

AverageTemperature
Country     Year                    
Afghanistan 1838           18.379571
            1839                 NaN
            1840           13.413455
            1841           13.997600
            1842           15.154667

In [201]:
df_year_temp.reset_index(inplace=True) # Let's reset the index so that we can use the Country and Year columns as a key to merge with the other datasets
df_year_temp.head()

,Country,Year,AverageTemperature
0,Afghanistan,1838,18.379571
1,Afghanistan,1839,NaN
2,Afghanistan,1840,13.413455
3,Afghanistan,1841,13.997600
4,Afghanistan,1842,15.154667


---

### Reading and cleaning the 3 others data sets
- GDP
- Population 
- Geopandas

#### GDP Data

In [202]:
df_gdp = pd.read_csv('..\data\gdp-data\GDP_Country.csv', skiprows=4) # Reading GDP_Country csv file
df_gdp.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,Unnamed: 66
0,Aruba,ABW,GDP (constant 2015 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,2.862306e+09,2.861720e+09,2.963128e+09,3.025850e+09,3.191738e+09,3.359555e+09,3.380889e+09,2.752412e+09,3.225070e+09,NaN
1,Africa Eastern and Southern,AFE,GDP (constant 2015 US$),NY.GDP.MKTP.KD,1.536964e+11,1.540611e+11,1.663621e+11,1.749528e+11,1.829721e+11,1.927209e+11,...,8.623341e+11,8.971645e+11,9.231439e+11,9.460928e+11,9.710653e+11,9.964178e+11,1.016728e+12,9.857923e+11,1.029191e+12,NaN
2,Afghanistan,AFG,GDP (constant 2015 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,1.918925e+10,1.971207e+10,1.999816e+10,2.045018e+10,2.099149e+10,2.124113e+10,2.207200e+10,2.155306e+10,1.708357e+10,NaN
3,Africa Western and Central,AFW,GDP (constant 2015 US$),NY.GDP.MKTP.KD,1.056755e+11,1.076147e+11,1.116749e+11,1.198082e+11,1.262691e+11,1.313913e+11,...,7.046760e+11,7.464664e+11,7.669580e+11,7.678299e+11,7.855332e+11,8.086763e+11,8.344802e+11,8.269667e+11,8.597592e+11,NaN
4,Angola,AGO,GDP (constant 2015 US$),NY.GDP.MKTP.KD,NaN,NaN,NaN,NaN,NaN,NaN,...,8.243377e+10,8.640707e+10,8.721930e+10,8.496904e+10,8.484159e+10,8.372481e+10,8.313874e+10,7.848297e+10,7.934628e+10,NaN


Drop unnecessary columns: 

In [203]:
df_gdp.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code','Unnamed: 66'], inplace=True)
df_gdp.rename(columns={"Country Name": "Country"}, inplace=True)
df_gdp.head()

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.689383e+09,2.862306e+09,2.861720e+09,2.963128e+09,3.025850e+09,3.191738e+09,3.359555e+09,3.380889e+09,2.752412e+09,3.225070e+09
1,Africa Eastern and Southern,1.536964e+11,1.540611e+11,1.663621e+11,1.749528e+11,1.829721e+11,1.927209e+11,2.002638e+11,2.107883e+11,2.192753e+11,...,8.273424e+11,8.623341e+11,8.971645e+11,9.231439e+11,9.460928e+11,9.710653e+11,9.964178e+11,1.016728e+12,9.857923e+11,1.029191e+12
2,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.817151e+10,1.918925e+10,1.971207e+10,1.999816e+10,2.045018e+10,2.099149e+10,2.124113e+10,2.207200e+10,2.155306e+10,1.708357e+10
3,Africa Western and Central,1.056755e+11,1.076147e+11,1.116749e+11,1.198082e+11,1.262691e+11,1.313913e+11,1.290167e+11,1.166363e+11,1.183228e+11,...,6.641073e+11,7.046760e+11,7.464664e+11,7.669580e+11,7.678299e+11,7.855332e+11,8.086763e+11,8.344802e+11,8.269667e+11,8.597592e+11
4,Angola,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.854575e+10,8.243377e+10,8.640707e+10,8.721930e+10,8.496904e+10,8.484159e+10,8.372481e+10,8.313874e+10,7.848297e+10,7.934628e+10


In [204]:
df_gdp = df_gdp.melt(id_vars=['Country'], var_name='Year', value_name='GDP') # Melt the dataset to have a single column for the years
df_gdp.head()

,Country,Year,GDP
0,Aruba,1960,NaN
1,Africa Eastern and Southern,1960,1.536964e+11
2,Afghanistan,1960,NaN
3,Africa Western and Central,1960,1.056755e+11
4,Angola,1960,NaN


In [205]:
df_gdp.sort_values(by=['Country'], inplace=True) # Order by country name
df_gdp.head()

,Country,Year,GDP
2396,Afghanistan,1969,NaN
4790,Afghanistan,1978,NaN
14632,Afghanistan,2015,1.999816e+10
15696,Afghanistan,2019,2.207200e+10
14100,Afghanistan,2013,1.918925e+10


#### Population data

In [206]:
df_pop = pd.read_csv('..\data\population-data\population_data.csv', skiprows=4) # Reading population_data csv file

Drop unnecesary columns:

In [207]:
df_pop.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code','Unnamed: 66'], inplace=True)
df_pop.rename(columns={"Country Name": "Country"}, inplace=True) # Rename Country column
df_pop.head()

,Country,1960,1961,1962,1963,1964,1965,1966,1967,1968,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Aruba,54608.0,55811.0,56682.0,57475.0,58178.0,58782.0,59291.0,59522.0,59471.0,...,102112.0,102880.0,103594.0,104257.0,104874.0,105439.0,105962.0,106442.0,106585.0,106537.0
1,Africa Eastern and Southern,130692579.0,134169237.0,137835590.0,141630546.0,145605995.0,149742351.0,153955516.0,158313235.0,162875171.0,...,552530654.0,567891875.0,583650827.0,600008150.0,616377331.0,632746296.0,649756874.0,667242712.0,685112705.0,702976832.0
2,Afghanistan,8622466.0,8790140.0,8969047.0,9157465.0,9355514.0,9565147.0,9783147.0,10010030.0,10247780.0,...,30466479.0,31541209.0,32716210.0,33753499.0,34636207.0,35643418.0,36686784.0,37769499.0,38972230.0,40099462.0
3,Africa Western and Central,97256290.0,99314028.0,101445032.0,103667517.0,105959979.0,108336203.0,110798486.0,113319950.0,115921723.0,...,376797999.0,387204553.0,397855507.0,408690375.0,419778384.0,431138704.0,442646825.0,454306063.0,466189102.0,478185907.0
4,Angola,5357195.0,5441333.0,5521400.0,5599827.0,5673199.0,5736582.0,5787044.0,5827503.0,5868203.0,...,25188292.0,26147002.0,27128337.0,28127721.0,29154746.0,30208628.0,31273533.0,32353588.0,33428486.0,34503774.0


In [208]:
df_pop = df_pop.melt(id_vars=['Country'], var_name='Year', value_name='Population') # Melt the dataset to have a single column for the years
df_pop.head()

,Country,Year,Population
0,Aruba,1960,54608.0
1,Africa Eastern and Southern,1960,130692579.0
2,Afghanistan,1960,8622466.0
3,Africa Western and Central,1960,97256290.0
4,Angola,1960,5357195.0


In [209]:
df_pop.sort_values(by=['Country'], inplace=True) # Order by country name
df_pop.head()

,Country,Year,Population
2396,Afghanistan,1969,10494489.0
4790,Afghanistan,1978,12938862.0
14632,Afghanistan,2015,33753499.0
15696,Afghanistan,2019,37769499.0
14100,Afghanistan,2013,31541209.0


#### Geopandas Dataset

In [210]:
world = geopandas.read_file(geopandas.datasets.get_path("naturalearth_lowres"))
print(world.head())

       pop_est      continent                      name iso_a3  gdp_md_est  \
0     889953.0        Oceania                      Fiji    FJI        5496   
1   58005463.0         Africa                  Tanzania    TZA       63177   
2     603253.0         Africa                 W. Sahara    ESH         907   
3   37589262.0  North America                    Canada    CAN     1736425   
4  328239523.0  North America  United States of America    USA    21433226   

                                            geometry  
0  MULTIPOLYGON (((180.00000 -16.06713, 180.00000...  
1  POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...  
2  POLYGON ((-8.66559 27.65643, -8.66512 27.58948...  
3  MULTIPOLYGON (((-122.84000 49.00000, -122.9742...  
4  MULTIPOLYGON (((-122.84000 49.00000, -120.0000...  


#### Merge the 4 dataframes: 

It is important to Verify that the types of the `Year` column are the same in both dataframes:

In [211]:
df_year_temp.dtypes

Country                object
Year                    int64
AverageTemperature    float64
dtype: object

In [212]:
df_gdp.dtypes

Country     object
Year        object
GDP        float64
dtype: object

Since it is not the same, we pass it the `Year` column to `int` type:

In [213]:
# Replace inconsisten year types with their standardized types in both dataframes
df_year_temp['Year'] = df_year_temp['Year'].astype(str)
df_gdp['Year'] = df_gdp['Year'].astype(str)

An important final approach in order to merge correctly the dataframes is to replace the `Country` names with their standarized names in all the dataframes.

In [214]:
df_year_temp['Country'] = df_year_temp['Country'].replace(country_dict)
df_gdp['Country'] = df_gdp['Country'].replace(country_dict)
world['name'] = world['name'].replace(country_dict)

In [215]:
df_merged = df_year_temp.merge(df_gdp, on=['Country', 'Year'], how='inner') # Merge the two dataframes
df_merged = df_merged.merge(df_pop, on=['Country', 'Year'], how='inner')
df_merged.head()

,Country,Year,AverageTemperature,GDP,Population
0,Afghanistan,1960,13.985417,NaN,8622466.0
1,Afghanistan,1961,14.064917,NaN,8790140.0
2,Afghanistan,1962,13.768667,NaN,8969047.0
3,Afghanistan,1963,15.033417,NaN,9157465.0
4,Afghanistan,1964,13.084917,NaN,9355514.0


In [216]:
# Save merge dataset in clean-data folder
df_merged.to_csv('..\data\clean-data\merged_data.csv', index=False)

## Calculations

In [217]:
# Lets study the percentage of missing values
df_merged.isnull().sum()/len(df_merged)

Country               0.000000
Year                  0.000000
AverageTemperature    0.000000
GDP                   0.252677
Population            0.000000
dtype: float64

In [218]:
missing_counts = df_merged['GDP'].isna().groupby(df_merged['Country']).sum()
missing_counts.sort_values(ascending=False)

Country
Liechtenstein             54
New Caledonia             54
British Virgin Islands    54
Faroe Islands             54
Somalia                   53
                          ..
Kenya                      0
Japan                      0
Italy                      0
Benin                      0
Zimbabwe                   0
Name: GDP, Length: 166, dtype: int64

In [219]:

total_counts = df_merged['GDP'].groupby(df_merged['Country']).count()

In [220]:

country_data = pd.DataFrame({
    'Country': missing_counts.index,
    'Missing_Count': missing_counts.values,
})
country_data.head()

,Country,Missing_Count
0,Afghanistan,42
1,Albania,20
2,Algeria,0
3,American Samoa,42
4,Andorra,10


In [221]:

sorted_data = country_data.sort_values('Missing_Count', ascending = False)
sorted_data.head(10)

,Country,Missing_Count
89,Liechtenstein,54
109,New Caledonia,54
23,British Virgin Islands,54
51,Faroe Islands,54
141,Somalia,53
42,Djibouti,53
30,Cayman Islands,46
0,Afghanistan,42
115,Northern Mariana Islands,42
3,American Samoa,42


Lets study with `Year` has less and the most missing data.

In [222]:
# Let´s study witch `Year` has the most missing values
df_merged['GDP'].isna().groupby(df_merged['Year']).sum().sort_values(ascending=False).head(5)

Year
1960    90
1962    88
1963    88
1964    88
1961    88
Name: GDP, dtype: int64

In [223]:

# Let´s study witch `Year` has the less missing values
df_merged['GDP'].isna().groupby(df_merged['Year']).sum().sort_values(ascending=True).head(5)

Year
2013    5
2011    6
2010    6
2009    6
2008    6
Name: GDP, dtype: int64

Since 2013 is the Year with less missing data we are going to plot that one:



In [224]:
df_2013 = pd.DataFrame(df_merged[df_merged.Year == "2013"])
df_2013.head()

,Country,Year,AverageTemperature,GDP,Population
53,Afghanistan,2013,16.533625,1.918925e+10,31541209.0
107,Albania,2013,14.993875,1.094547e+10,2895092.0
161,Algeria,2013,25.121500,1.541977e+11,38000626.0
215,American Samoa,2013,27.517250,6.411481e+08,52995.0
269,Andorra,2013,12.307875,2.683225e+09,71367.0


> **Note:** We will use the 2013 data to make our analysis:

In [236]:
df_2013_pop = df_2013[['Country', 'Population']] # 2013 Population
df_2013_gdp = df_2013[['Country', 'GDP']] # 2013 GDP
df_2013_temp = df_2013[['Country', 'AverageTemperature']] # 2013 Average Temperature


### Temperature Plot 2013



In [226]:
df_2013_temp.head()

,Country,AverageTemperature
53,Afghanistan,16.533625
107,Albania,14.993875
161,Algeria,25.121500
215,American Samoa,27.517250
269,Andorra,12.307875


In [227]:
data_map_2013_temp = world.merge(df_2013_temp, how = "left", left_on=["name"], right_on=["Country"]) # World map with 2013 Average Temperature
data_map_2013_temp.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,Country,AverageTemperature
0,889953.0,Oceania,Fiji,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,25.941375
1,58005463.0,Africa,Tanzania,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",Tanzania,22.772750
2,603253.0,Africa,W. Sahara,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",NaN,NaN
3,37589262.0,North America,Canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",Canada,-1.640667
4,328239523.0,North America,United States of America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",NaN,NaN


In [228]:
my_map = folium.Map()

folium.Choropleth(
    geo_data=data_map_2013_temp,
    name='choropleth',
    data=data_map_2013_temp,
    columns=['Country', 'AverageTemperature'],
    key_on='feature.properties.name',
    fill_color= 'Reds',
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name='Average Temperature by Country',
    tooltip=folium.features.GeoJsonTooltip(fields=['name', 'AverageTemperature'], 
        aliases=['Country', 'Average Temperature'], 
        labels=True, 
        sticky=False, 
        toLocaleString=True),
    # gradient_fill=color_scale
).add_to(my_map)

# Add the layer control to the map
folium.LayerControl().add_to(my_map)

# Save the map to an HTML file
my_map.save('..\ 2013_temp.html')

### GDP map 2013

In [229]:
df_2013_gdp.head()

,Country,GDP
53,Afghanistan,1.918925e+10
107,Albania,1.094547e+10
161,Algeria,1.541977e+11
215,American Samoa,6.411481e+08
269,Andorra,2.683225e+09


In [230]:
data_map_2013_gdp = world.merge(df_2013_gdp, how = "left", left_on=["name"], right_on=["Country"]) # World map with 2013 Average Temperature
data_map_2013_gdp.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,Country,GDP
0,889953.0,Oceania,Fiji,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,4.243092e+09
1,58005463.0,Africa,Tanzania,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",Tanzania,4.181405e+10
2,603253.0,Africa,W. Sahara,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",NaN,NaN
3,37589262.0,North America,Canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",Canada,1.503174e+12
4,328239523.0,North America,United States of America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",NaN,NaN


In [231]:
#Creating a threshold scale
n = 250 # number of elements in the vector
diff = (data_map_2013_gdp['GDP'].max() - data_map_2013_gdp['GDP'].min()) / n # fixed difference between elements
threshold_scale = list(np.arange(data_map_2013_gdp['GDP'].min(), data_map_2013_gdp['GDP'].max() + diff, diff))

my_map = folium.Map()

folium.Choropleth(
    geo_data=data_map_2013_gdp,
    name='choropleth',
    data=data_map_2013_gdp,
    columns=['Country', 'GDP'],
    key_on='feature.properties.name',
    fill_color= 'Spectral',
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name=f"GDP range: {data_map_2013_gdp['GDP'].min():,.2f} to {data_map_2013_gdp['GDP'].max():,.2f}",
    tooltip=folium.features.GeoJsonTooltip(fields=['name', 'GDP'], 
        aliases=['Country', 'GDP'], 
        labels=False, 
        sticky=False, 
        toLocaleString=True),
    threshold_scale=threshold_scale,
    legend_format='',
).add_to(my_map)

# Add the layer control to the map
folium.LayerControl().add_to(my_map)

# Save the map to an HTML file
my_map.save('..\ 2013_gdp.html')

### Population map 2013

In [232]:
df_2013_pop.head()

,Country,Population
53,Afghanistan,31541209.0
107,Albania,2895092.0
161,Algeria,38000626.0
215,American Samoa,52995.0
269,Andorra,71367.0


In [238]:
data_map_2013_pop = world.merge(df_2013_pop, how = "left", left_on=["name"], right_on=["Country"]) # World map with 2013 Population
data_map_2013_pop.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry,Country,Population
0,889953.0,Oceania,Fiji,FJI,5496,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000...",Fiji,913453.0
1,58005463.0,Africa,Tanzania,TZA,63177,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982...",Tanzania,49253643.0
2,603253.0,Africa,W. Sahara,ESH,907,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948...",NaN,NaN
3,37589262.0,North America,Canada,CAN,1736425,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742...",Canada,35082954.0
4,328239523.0,North America,United States of America,USA,21433226,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000...",NaN,NaN


In [242]:
#Creating a threshold scale
n = 250 # number of elements in the vector
diff = (data_map_2013_pop['Population'].max() - data_map_2013_pop['Population'].min()) / n # fixed difference between elements
threshold_scale = list(np.arange(data_map_2013_pop['Population'].min(), data_map_2013_pop['Population'].max() + diff, diff))

my_map = folium.Map()

folium.Choropleth(
    geo_data=data_map_2013_pop,
    name='choropleth',
    data=data_map_2013_pop,
    columns=['Country', 'Population'],
    key_on='feature.properties.name',
    fill_color= 'Spectral',
    fill_opacity=0.8,
    line_opacity=0.2,
    legend_name=f"Population range: {data_map_2013_pop['Population'].min():,.2f} to {data_map_2013_pop['Population'].max():,.2f}",
    tooltip=folium.features.GeoJsonTooltip(fields=['name', 'Population'], 
        aliases=['Country', 'Population'], 
        labels=False, 
        sticky=False, 
        toLocaleString=True),
    threshold_scale=threshold_scale,
    legend_format='',
).add_to(my_map)

# Add the layer control to the map
folium.LayerControl().add_to(my_map)

# Save the map to an HTML file
my_map.save('..\ 2013_pop.html')

Displaying the 3 diferent maps from 2013

In [245]:
from IPython.display import IFrame
IFrame(src='../2013_pop.html', width=700, height=600)

### Population map 2013



### Combining and cleaning data



### Calculations